In [1]:
%matplotlib inline

In [2]:
import torch 
import torchvision.transforms as transforms
from torchvision import datasets, models
import numpy as np
import torchvision.models
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import time
import os

In [3]:
def load_split_train_val(datadir, valid_size = .25):

    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225])
    train_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                       transforms.ToTensor(),
                                       normalize
                                       ])
    val_transforms = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      normalize
                                      ])
    train_data = torchvision.datasets.ImageFolder(datadir,
                    transform=train_transforms)
    val_data = torchvision.datasets.ImageFolder(datadir,
                    transform=val_transforms)
    num_train = len(train_data)
    indices = list(range(num_train))
    classes = train_data.classes
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, val_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=1)
    valloader = torch.utils.data.DataLoader(val_data,
                   sampler=val_sampler, batch_size=1)
    dataiter = iter(trainloader) 
    images, labels = dataiter.next()
    return trainloader, valloader, classes, labels

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=5):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0
    #train_losses, test_losses = [], []

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            i = 0
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
                dataloader = trainloader
            else:
                model.train(False)  # Set model to evaluate mode
                dataloader = valloader
            running_loss = 0.0
            running_corrects = 0.0

            # Iterate over data.
            for data in dataloader:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                
                running_loss += loss.data
                running_corrects += torch.sum(preds == labels.data).to(torch.float32)
                #print (str(i) + ":" + str(running_loss) + ":" + str(running_corrects))
                #i += 1
                
                #train_losses.append(running_loss/len(trainloader))
                #test_losses.append(test_loss/len(testloader)) 
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model#, train_losses, test_losses

In [6]:
data_dir = 'data/masked_images/data/trainval/breeds_cat/'
trainloader, valloader, classes, labels = load_split_train_val(data_dir, .25)

In [7]:
dataset_sizes = {
    "train": len(trainloader),
    "val": len(valloader)
}

# use gpu or not
use_gpu = torch.cuda.is_available()

model_cat = torchvision.models.resnet50(pretrained=True)
#model_cat = torchvision.models.vgg16(pretrained=True)s
num_ftrs = model_cat.fc.in_features
#model_cat.classifier[-1] = nn.Linear(in_features=4096, out_features=12)

model_cat.fc = nn.Linear(num_ftrs, 12)

if use_gpu:
    model_cat = model_cat.cuda()

# define loss function
criterion = nn.CrossEntropyLoss()


# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_cat.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)


In [8]:
model_cat = train_model(model=model_cat,
                           criterion=criterion,
                           optimizer=optimizer_ft,
                           scheduler=exp_lr_scheduler,
                           num_epochs=15)
torch.save(model_cat, './resnet50_breed_cat.pth')

Epoch 0/14
----------
train Loss: 2.5813 Acc: 0.0839
val Loss: 2.5337 Acc: 0.1342
Epoch 1/14
----------
train Loss: 2.5726 Acc: 0.0846
val Loss: 2.4265 Acc: 0.1405
Epoch 2/14
----------
train Loss: 2.5520 Acc: 0.0964
val Loss: 2.3451 Acc: 0.1426
Epoch 3/14
----------
train Loss: 2.5368 Acc: 0.0936
val Loss: 2.2183 Acc: 0.1950
Epoch 4/14
----------
train Loss: 2.4951 Acc: 0.1034
val Loss: 2.1274 Acc: 0.3375
Epoch 5/14
----------
train Loss: 2.3836 Acc: 0.1516
val Loss: 1.9724 Acc: 0.3878
Epoch 6/14
----------
train Loss: 2.2519 Acc: 0.1775
val Loss: 1.8092 Acc: 0.3753
Epoch 7/14
----------
train Loss: 2.0518 Acc: 0.3249
val Loss: 1.6988 Acc: 0.5388
Epoch 8/14
----------
train Loss: 2.0110 Acc: 0.4116
val Loss: 1.6851 Acc: 0.5430
Epoch 9/14
----------
train Loss: 1.9882 Acc: 0.4088
val Loss: 1.7171 Acc: 0.5241
Epoch 10/14
----------
train Loss: 1.9654 Acc: 0.4067
val Loss: 1.5770 Acc: 0.4864
Epoch 11/14
----------
train Loss: 1.9394 Acc: 0.4095
val Loss: 1.5772 Acc: 0.5891
Epoch 12/14
--

In [9]:
test_data_dir = 'data/masked_images/data/test/breeds_cat/'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=torch.load('resnet50_breed_cat.pth')
test_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                     ])
def predict_image(image):
    image_tensor = test_transforms(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input = Variable(image_tensor)
    input = input.to(device)
    output = model(input)
    index = output.data.cpu().numpy().argmax()
    return index
def get_random_images(num):
    data = datasets.ImageFolder(test_data_dir, transform=test_transforms)
    classes = data.classes
    indices = list(range(len(data)))
    np.random.shuffle(indices)
    idx = indices[:num]
    from torch.utils.data.sampler import SubsetRandomSampler
    sampler = SubsetRandomSampler(idx)
    loader = torch.utils.data.DataLoader(data, 
                   sampler=sampler, batch_size=num)
    dataiter = iter(loader)
    images, labels = dataiter.next()
    return images, labels, classes

In [ ]:

to_pil = transforms.ToPILImage()
images, labels, classes = get_random_images(5)
fig=plt.figure(figsize=(20,20))
for ii in range(len(images)):
    image = to_pil(images[ii])
    index = predict_image(image)
    sub = fig.add_subplot(1, len(images), ii+1)
    res = int(labels[ii]) == index
    sub.set_title(str(classes[index]) + ":" + str(res))
    plt.axis('off')
    plt.imshow(image)
plt.show()

In [15]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225])
test_transforms1 = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                        normalize
                                     ])
test_data = torchvision.datasets.ImageFolder(test_data_dir,
                    transform=test_transforms1)
testloader = torch.utils.data.DataLoader(test_data,
                 batch_size=1)
total_images = 0
total_correct = 0
class_correct = list(0. for i in range(12))
class_total = list(0. for i in range(12))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        if use_gpu:
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
        else:
            images, labels = Variable(inputs), Variable(labels)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        label = labels[0]
        class_correct[label] += c.item()
        class_total[label] += 1
        total_images += 1
        total_correct += c.item()
print('Overall accuracy for all breeds %2d%% ' % (
            100 * total_correct / total_images))
print() 
for i in range(12):
        print('Accuracy of %5s : %2d %%' % (
            
            classes[i], 100 * class_correct[i] / class_total[i]))

Overall accuracy for all breeds 46% 

Accuracy of Abyssinian : 52 %
Accuracy of Bengal :  5 %
Accuracy of Birman : 47 %
Accuracy of Bombay : 46 %
Accuracy of British_Shorthair : 75 %
Accuracy of Egyptian_Mau :  0 %
Accuracy of Maine_Coon : 87 %
Accuracy of Persian : 67 %
Accuracy of Ragdoll : 10 %
Accuracy of Russian_Blue :  0 %
Accuracy of Siamese : 75 %
Accuracy of Sphynx : 82 %
